# 시퀀스(sequences)
- 스트림과 유사한 기능을 제공
- Kotlin에서도 동작할 수 있도록 독자적으로 개발

## 1. 시퀀스가 필요한 이유
- 대용량 컬렉션 처리
- 크기를 알 수 없는 데이터 처리
> 작은 데이터 일경우 일반 컬렉션이 더 효율 좋음
### 중간컬렉션 생성 문제
컬렉션 연산을 체이닝할 때 각 단계마다 중간 컬렉션이 생성
- 작은 컬렉션에서는 문제가 되지 않지만, 수십만 개의 아이템을 처리할 대는 메모리 효율성 문제가 발생할 수 있음.
```kotlin
// 예제: Ford 자동차의 색상 찾기
cars.filter { it.model == "Ford" }  // 중간 컬렉션 생성
    .map { it.color }               // 또 다른 중간 컬렉션 생성
```

## 2. 시퀀스 작동 원리
- 자바에서 컬렉션: 각 단계에서 전체 컬렉션을 한 번에 평가하고 결과를 다음 단계로 전달
- 시퀀스: 각 요소를 개별적으로 평가하고 체인의 다음단계로 전달
### 생성 방법

In [ ]:
val sequence = collection.asSequence()

## 3. 중간 연산과 터미널 연산
### 중간 연산
- 다른 시퀀스를 반환
- 지연 평가(lazy evaluation) 방식
- filter, map, take 등등
> 지연 평가: 실제로 결과가 필요할 때까지 연산을 미루는 방식. 시퀀스에서는 터미널 연산이 호출되기 전까지 중간 연산들이 실행되지 않습니다.
### 터미널 연산
- 시퀀스가 아닌 구체적인 결과를 반환
- 체인을 종료 시킴
- toList, find, count


In [6]:
val names = listOf("Joe", "Mary", "Jane")
val result = names.asSequence()
    .filter {
        println("filtering $it")
        it.startsWith("J")
    }
    .map {
        println("mapping $it")
        it.uppercase()
    }
// 이 시점에서는 아무것도 실행되지 않음

println("시퀀스 정의 완료")


val list = result.toList() // 터미널 연산에서 비로소 실행


시퀀스 정의 완료


### 왜 이런 일이 발생하는가?

- `중간 연산의 본질`: filter와 map은 중간 연산으로, 실제 작업을 수행하지 않고 "무엇을 해야 할지"만 기록합니다.
- `시퀀스 객체의 역할`: result 변수는 실제 데이터가 아니라 "연산 계획서"를 담고 있는 시퀀스 객체입니다.
- `터미널 연산의 트리거`: toList()가 호출되는 순간, 시퀀스는 실제로 각 요소를 순회하며 정의된 연산들을 실행합니다.


### 지연 평가 장점
1. 메모리 효율성: 중간 결과를 저장할 컬렉션을 생성하지 않음
2. 조기 종료 가능: 필요한 만큼만 처리하고 멈출 수 있음
3. 성능 최적화: 불필요한 연산을 피할 수 있음

In [8]:
val names = listOf("Joe", "Mary", "Jane")
val result = names.asSequence()
    .filter { it.startsWith("J") }
    .map { it.uppercase() }
    .find { it.endsWith("E") }

// 실제 실행 순서:
// 1. "Joe" → filter 통과 → map으로 "JOE" → find에서 "E"로 끝남 → 결과 반환!
// 2. "Mary", "Jane"은 아예 평가되지 않음, 맨 처음 찾았으# (조기 종료)
println(result)

JOE


### 조기 종료 장점
- find() 같은 터미널 연산을 사용할 때, 조건을 만족하는 첫 번째 요소를 찾으면 즉시 종료

### 연산 순서 중요성

In [ ]:
// 비효율적: 모든 요소를 먼저 변환한 후 필터링
names.map { it.uppercase() }
    .filter { it.startsWith("J") }

// 효율적: 불필요한 요소를 먼저 제거한 후 변환
names.filter { it.startsWith("J") }
    .map { it.uppercase() }